# Arrays and views

Julia has excellent functionality for manipulating $N$-dimensional arrays. We will have a quick look at this subject (which is more complicated than you might suspect). 

Note that in Julia 0.7 there are significant changes, especially to how transposes work, and how different types of manipulations of arrays look.

Let's define a $3 \times 3$ array (matrix):

In [ ]:
M = [1 2 3; 4 5 6; 7 8 9]  # a 3x3 matrix

In [ ]:
typeof(M)

We can extract part of the matrix using indexing notation:

In [ ]:
part = M[2:3, 1:2]

What happens if we modify `part`?

In [ ]:
part[1, 1]

In [ ]:
part[1, 1] = 10

In [ ]:
part

In [ ]:
M

We see that `M` has *not* been modified; that means that `part` was a **copy** of that part of `M`.

## Views

We often do *not* want a copy, but rather just a reference to the same data, which is called a `view`: 

In [ ]:
V = view(M, 2:3, 1:2)

In [ ]:
typeof(V)

Although this type looks (and is) somewhat complicated, it just contains the necessary information for the object to manipulate correctly the underlying data. To decompose this type expression, see the array types notebook.

In [ ]:
V isa AbstractArray

In [ ]:
subtypes(AbstractArray)  # behave like a generalized Array

If we modify `V`, then `M` also gets modified, since it is the same data:

In [ ]:
V

In [ ]:
show(V)

In [ ]:
V[1, 1]

In [ ]:
V[1, 1] = 100

In [ ]:
V

In [ ]:
M

If we have a complicated expression, we can use the `@view` macro to make the syntax nicer; this turns an indexing operation into the corresponding `view`:

In [ ]:
@view M[2:3, 1:2]

To turn every indexing operation in a block of code into a view, use `@views`.

## In-place and vectorized operations: "`.`" ("pointwise")

Suppose we have two matrices and wish to add one to the other:

In [ ]:
N = 10
A = rand(N, N)
B = rand(N, N);

Coming from other languages, we might expect to be able to write `A += B`, and indeed this works:

In [ ]:
A += B

In [ ]:
expand(:(A += B))

We see that this is just "syntactic sugar" (i.e. a cute way of writing) `A = A + B`.

However, it turns out that this does not do what you might think it does, namely "in-place addition", in which each element of `A` is updated in place. Rather, it allocates a new temporary object for the result of `A + B`. We can see this:

In [ ]:
using BenchmarkTools

N = 1000
A = rand(N, N)
B = rand(N, N)

@btime $A += $B;

Note the large amount of allocation here (1,000,000 $\times$ 8 bytes).

The in-place behaviour can be obtained using **pointwise operators** (with `.`); this is called **broadcasting**:

In [ ]:
A .= A .+ B

In [ ]:
@btime $A .= $A .+ $B;  # no allocations

Furthermore, we can chain such operations together with no creation of temporaries:

In [ ]:
C = rand(1000, 1000)

@btime A .+= B + C;  # allocates

In [ ]:
@btime $A .+= $B .+ $C  # does not allocate  

This is equivalent to

In [ ]:
for i in eachindex(A)
    A[i] += B[i] + C[i]
end

See [this blog post by Steven Johnson](https://julialang.org/blog/2017/01/moredots) for more details.

There is a `@.` macro for "pointing" every operation:

In [ ]:
@btime @. $A += $B + $C

## Efficient small matrices and vectors

For small matrices and vectors, the generic vector and matrix code is too slow, since the type does not contain the information on the number of elements contained in the array, so that generic loops are used.

The `StaticArrays.jl` package fixes this problem by **unrolling** operations for small arrays.

In [ ]:
# Pkg.add("StaticArrays")

using StaticArrays, BenchmarkTools

In [ ]:
function bench()
    x = SVector(1, 2)
    y = [1, 2]
    
    @btime $x + $x
    @btime $y + $y
end

In [ ]:
bench()

In [ ]:
x = SVector(1, 2)
@code_lowered x + x

In [ ]:
@code_typed x + x

In [ ]:
@code_llvm x + x

In [ ]:
@code_native x + x

In [ ]:
y = [1, 2]
@code_native y + y